# Purpose

Analyze the collated results from the 1-D fine-tuning experiments. See that notebook [here](https://github.com/myles-i/DLH_TransferLearning/blob/master/jupyter_notebooks/collate_1d_finetune_results.ipynb).

## Background

The three weight type scenarios are: 10%, 20%, and random. Weight refers to the weights of the ResNet-18v2 at the commencement of fine-tuning.

For each weight-type, we ran fine-tuning 10 times with 10 different seeds. The seed plays a role in selecting the validation set. For all three scenarios, we use the same 10 seeds so that we can get more apples-to-apples comparison between trials between different scenarios.

We use google drive mount in this notebook, but in the final report we will obtain the result files using `gdown`.



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
PROJECT_DIR = '/content/drive/MyDrive/DLHProject'
RESULT_DIR = PROJECT_DIR + '/results1d'

In [4]:
! ls -l {RESULT_DIR}

total 1753
-rw------- 1 root root  192825 Apr 26 07:04 history_all.csv
-rw------- 1 root root 1601402 Apr 26 07:20 predictions_all.pkl


# History Analysis

We analyze validation F1 scores and look at the effects of using 10% and 20% pre-train weights on fine-tuning results.

In [5]:
import pandas as pd

In [6]:
history_all = pd.read_csv(RESULT_DIR + '/history_all.csv')

### Epochs per weight type

Average epochs of each experiment (seed) for a given scenario, for each scenario.

We see that both pre-training scenarios take fewer epochs to complete fine-tuning compared to the random scenario. The 10% scenario was slightly faster compared to the 20% scenario in fine-tuning.

In [171]:
def make_epoch_table(history_all, decimals=1):
    table = (
        history_all.groupby(['weight_type', 'seed'])
        .max()
        .groupby('weight_type')
        .agg(
            Mean=pd.NamedAgg('epoch', 'mean'),
            Std=pd.NamedAgg('epoch', 'std'),
        )
        .round(decimals)
        .reindex(['random', '10', '20'])
        .rename(index={
            'random': 'Random',
            '10': 'Pre-train 10',
            '20': 'Pre-train 20'
        })
        .reset_index()
        .rename(columns={
            'weight_type': 'Scenario',
        })
    )
    # Explicitly select columns in desired order.
    table = table[['Scenario', 'Mean', 'Std']]
    return table

make_epoch_table(history_all)

,Scenario,Mean,Std
0,Random,71.0,5.6
1,Pre-train 10,56.2,2.1
2,Pre-train 20,56.8,2.6


### Macro F1 statistics per weight type

For `load_pkl` function, we copy its implementation directly from the [source](https://github.com/myles-i/DLH_TransferLearning/blob/master/transplant/utils.py#L183) as it's simple and also this notebook is kind of a 'scratch' one so we don't bother with git cloning the project repo in an effort to move quickly.

In [31]:
# Adapted from Weimann, et al source code.
import gzip
import pickle

def load_pkl(file, compress=True):
    """ Load pickled file. """
    if compress:
        with gzip.open(file, 'rb') as fh:
            return pickle.load(fh)

In [32]:
predictions_all = load_pkl(RESULT_DIR + '/predictions_all.pkl')

In [34]:
from sklearn.metrics import f1_score
import numpy as np

def my_f1(y_true, y_prob, average='macro'):
    # set average=None to get per-class F1 scores.
    y_pred = y_prob >= np.max(y_prob, axis=1)[:, None]
    return f1_score(y_true, y_pred, average=average)

In [145]:
def make_macro_f1_table(predictions):
    macro = []
    for key, value in predictions.items():
        weight_type, seed = key.split('_')
        identifier = {'weight_type': weight_type, 'seed': seed}
        macro_f1 = my_f1(value['y_true'], value['y_prob'], average='macro')
        macro_f1 = {'f1': macro_f1}
        macro.append(identifier | macro_f1)

    macro_scores = pd.DataFrame(macro)
    macro_df = macro_scores.groupby(['weight_type'])[['f1']].agg(
        mean=pd.NamedAgg('f1', 'mean'),
        std=pd.NamedAgg('f1', 'std'),
    )
    macro_df['macro_f1'] = make_mean_std_column(
        macro_df['mean'], macro_df['std']
    )
    macro_df = macro_df[['macro_f1']]
    return macro_df

In [ ]:
def make_per_class_f1_table(predictions):
    per_class = []
    for key, value in predictions.items():
        weight_type, seed = key.split('_')
        identifier = {'weight_type': weight_type, 'seed': seed}

        class_f1 = my_f1(value['y_true'], value['y_prob'], average=None)
        class_f1 = dict(zip(value['classes'], class_f1))
        per_class.append(identifier | class_f1)

    per_class_scores = pd.DataFrame(per_class)
    per_class_macro = (
        per_class_scores.groupby(['weight_type'])[['N', 'A', 'O', '~']]
        .agg(**{
            'N_mean': pd.NamedAgg('N', 'mean'),
            'A_mean': pd.NamedAgg('A', 'mean'),
            'O_mean': pd.NamedAgg('O', 'mean'),
            '~_mean': pd.NamedAgg('~', 'mean'),
            'N_std': pd.NamedAgg('N', 'std'),
            'A_std': pd.NamedAgg('A', 'std'),
            'O_std': pd.NamedAgg('O', 'std'),
            '~_std': pd.NamedAgg('~', 'std'),
        })
    )

    per_class_macro['N_f1'] = make_mean_std_column(
        per_class_macro['N_mean'], per_class_macro['N_std']
    )
    per_class_macro['A_f1'] = make_mean_std_column(
        per_class_macro['A_mean'], per_class_macro['A_std']
    )
    per_class_macro['O_f1'] = make_mean_std_column(
        per_class_macro['O_mean'], per_class_macro['O_std']
    )
    per_class_macro['~_f1'] = make_mean_std_column(
        per_class_macro['~_mean'], per_class_macro['~_std'],
    )
    per_class_macro = per_class_macro[['N_f1', 'A_f1', 'O_f1', '~_f1']]
    return per_class_macro

In [173]:
def make_mean_std_column(mean_col, std_col, decimals=3):
    """
    Helper to produce column values formatted like in Table 1 of the paper.
    """
    # Account for decimal, and assume the leading zero is stripped.
    return (
        mean_col.astype(str)
        # 1: decimal point's index, e.g.: '0.xxx'
        # 1 + decimals + 1: (decimals + 1) is total string length, counting
        # the decimal point itself. The (1 +) is because we started the
        # substring at index 1.
        .str[1 : 1 + decimals + 1]
        .str.pad(decimals + 1, side='right', fillchar='0')
        # TODO: may need to replace \u00b1 with '+/-' if it doesn't render
        # correctly in nbconvert
        + ' (\u00b1 '
        + std_col.astype(str)
        .str[1 : 1 + decimals + 1]
        .str.pad(decimals + 1, side='right', fillchar='0')
        + ')'
    )

In [172]:
def make_f1_table(predictions):
    macro_f1 = make_macro_f1_table(predictions)
    per_class_f1 = make_per_class_f1_table(predictions)
    table = (
        macro_f1.join(per_class_f1)
        .reindex(['random', '10', '20'])
        .rename(index={
            'random': 'Random',
            '10': 'Pre-train 10',
            '20': 'Pre-train 20',
        })
        .reset_index()
        # Using same names as Table 1
        .rename(columns={
            'weight_type': 'Scenario',
            'macro_f1': 'F1',
            'N_f1': 'F1n', # Normal
            'A_f1': 'F1a', # AF
            'O_f1': 'F1o', # Other
            '~_f1': 'F1p', # Noisy
        })
    )
    # Explicitly select columns in desired order.
    table = table[['Scenario', 'F1', 'F1n', 'F1a', 'F1o', 'F1p']]
    return table

make_f1_table(predictions_all)

,Scenario,F1,F1n,F1a,F1o,F1p
0,Random,.702 (± .030),.881 (± .011),.647 (± .048),.686 (± .030),.595 (± .063)
1,Pre-train 10,.765 (± .012),.904 (± .005),.745 (± .017),.735 (± .013),.675 (± .030)
2,Pre-train 20,.765 (± .014),.901 (± .004),.760 (± .026),.741 (± .012),.658 (± .032)
